### 基准模型

In [12]:
from __future__ import print_function
from six.moves import cPickle as pickle

pickle_file = 'svhn_single.pickle'

with open(pickle_file, 'rb') as f:
    dataset = pickle.load(f)
    train_dataset = dataset['train_dataset']
    train_labels = dataset['train_labels']
    valid_dataset = dataset['valid_dataset']
    valid_labels = dataset['valid_labels']
    test_dataset = dataset['test_dataset'][:10000]
    test_labels = dataset['test_labels'][:10000]
    del dataset
    
print('Training dataset shape: ', train_dataset.shape)
print('Training labels shape: ', train_labels.shape)
print('Validing dataset shape: ', valid_dataset.shape)
print('Validing labels shape: ', valid_labels.shape)
print('Testing dataset shape: ', test_dataset.shape)
print('Testing labels shape: ', test_labels.shape)

Training dataset shape:  (72525, 32, 32)
Training labels shape:  (72525,)
Validing dataset shape:  (732, 32, 32)
Validing labels shape:  (732,)
Testing dataset shape:  (10000, 32, 32)
Testing labels shape:  (10000,)


In [14]:
IMAGE_SIZE = 32
NUM_LABELS = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, IMAGE_SIZE*IMAGE_SIZE*num_channels)).astype(np.float32)
    #labels = (np.array([10,1,2,3,4,5,6,7,8,9]) == labels.reshape(-1, 1)).astype(np.float32) # one-hot encoding
    labels[labels==10] = 0
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training dataset shape: ', train_dataset.shape)
print('Training labels shape: ', train_labels.shape)
print('Validing dataset shape: ', valid_dataset.shape)
print('Validing labels shape: ', valid_labels.shape)
print('Testing dataset shape: ', test_dataset.shape)
print('Testing labels shape: ', test_labels.shape)

Training dataset shape:  (72525, 1024)
Training labels shape:  (72525,)
Validing dataset shape:  (732, 1024)
Validing labels shape:  (732,)
Testing dataset shape:  (10000, 1024)
Testing labels shape:  (10000,)


In [15]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(train_dataset, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [23]:
from sklearn.linear_model import SGDClassifier

classifier = SGDClassifier(loss='hinge')
classifier.fit(train_dataset, train_labels)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

逻辑回归分类器  
分类准确率 0.2364

SVM分类器  
分类准确率 0.127

In [24]:
from sklearn.metrics import accuracy_score

test_prediction = classifier.predict(test_dataset)
print('Test accuracy', accuracy_score(test_prediction, test_labels))

Test accuracy 0.127
